In [2]:
# ============================================================
# 🏙️ Analyse Immobilière – DVF Île-de-France (Préparation)
# ============================================================

# 📦 Imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import requests
from pathlib import Path
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

# ---------------- CONFIG ----------------
DATA_PATH = Path("./data/dvf_idf_clean.csv")
DATA_DIR = Path("./data")
COMMUNES_CSV = DATA_DIR / "communes_insee.csv"
DATA_DIR.mkdir(exist_ok=True)

# ---------------- 1️⃣ Téléchargement automatique des communes ----------------
def ensure_communes_csv():
    """Télécharge la correspondance code INSEE ↔ nom commune pour l'Île-de-France."""
    if COMMUNES_CSV.exists():
        display(Markdown("✅ **communes_insee.csv déjà présent.**"))
        return

    display(Markdown("📥 Création du fichier `communes_insee.csv`..."))
    dept_ids = ["75","77","78","91","92","93","94","95"]
    frames = []

    for dep in dept_ids:
        url = f"https://geo.api.gouv.fr/communes?codeDepartement={dep}&fields=nom,code&format=json"
        try:
            r = requests.get(url, timeout=20)
            r.raise_for_status()
            js = r.json()
            frames.append(pd.DataFrame([{"code_insee": it["code"], "nom_commune": it["nom"]} for it in js]))
        except Exception as e:
            print(f"⚠️ Impossible de récupérer les communes du département {dep} : {e}")

    if not frames:
        raise RuntimeError("❌ Aucune commune récupérée depuis geo.api.gouv.fr")

    ref = pd.concat(frames, ignore_index=True).drop_duplicates("code_insee")
    ref["code_insee"] = ref["code_insee"].astype(str)
    ref["nom_commune"] = ref["nom_commune"].astype(str).str.upper().str.strip()
    ref.to_csv(COMMUNES_CSV, index=False, encoding="utf-8")
    display(Markdown(f"✅ Fichier sauvegardé : `{COMMUNES_CSV}`"))

ensure_communes_csv()

# ---------------- 2️⃣ Chargement du fichier DVF + jointure ----------------
def load_dvf():
    df = pd.read_csv(DATA_PATH, sep="|", low_memory=False, dtype=str)

    for col in ["valeurfonc", "sbati"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df = df.dropna(subset=["valeurfonc", "sbati"])
    df = df[df["sbati"] > 8]
    df["prix_m2"] = df["valeurfonc"] / df["sbati"]

    # Jointure avec communes
    communes = pd.read_csv(COMMUNES_CSV, dtype=str)
    communes.columns = [c.lower().strip() for c in communes.columns]

    df["code_insee_clean"] = (
        df["l_codinsee"]
        .astype(str)
        .str.replace(r"\s+", "", regex=True)
        .str.split(",").str[0]
        .str.extract(r"(\d{5})")[0]
    )

    df = df.merge(
        communes[["code_insee", "nom_commune"]],
        left_on="code_insee_clean",
        right_on="code_insee",
        how="left"
    )

    df["nom_commune"] = df["nom_commune"].fillna(df["code_insee_clean"])
    df["nom_commune"] = df["nom_commune"].astype(str).str.upper().str.strip()

    if "datemut" in df.columns:
        df["datemut"] = pd.to_datetime(df["datemut"], errors="coerce")
        df["annee"] = df["datemut"].dt.year
    else:
        df["annee"] = None

    return df

df = load_dvf()
display(Markdown("✅ **Fichier DVF chargé avec succès !**"))
display(df.head())


✅ **communes_insee.csv déjà présent.**

✅ **Fichier DVF chargé avec succès !**

,idmutation,idmutinvar,idopendata,idnatmut,codservch,refdoc,datemut,anneemut,moismut,coddep,...,smai5pp,codtypbien,libtypbien,geompar_x,geompar_y,prix_m2,code_insee_clean,code_insee,nom_commune,annee
0,49116809,a1f3e33bd434de481330c6b33455c5e3,a1f3e33bd434de481330c6b33455c5e3,1,NaN,NaN,2014-10-13,2014,10,92,...,0.0,121,UN APPARTEMENT,648157.5987993651,6866395.463419259,7560.000000,92044,92044,LEVALLOIS-PERRET,2014
1,49595837,a8400660cc213499f774b168d313e19d,a8400660cc213499f774b168d313e19d,1,NaN,NaN,2014-02-17,2014,2,92,...,0.0,121,UN APPARTEMENT,645335.7350825083,6859572.6063931035,7500.000000,92012,92012,BOULOGNE-BILLANCOURT,2014
2,49600265,8dff60e19651ece56e8509320bff731d,8dff60e19651ece56e8509320bff731d,1,NaN,NaN,2014-02-07,2014,2,92,...,0.0,121,UN APPARTEMENT,647685.0227941211,6867697.0722600445,6000.000000,92004,92004,ASNIÈRES-SUR-SEINE,2014
3,49757741,9ce0798d37b91bc749a6f6602976518a,9ce0798d37b91bc749a6f6602976518a,1,NaN,NaN,2014-12-19,2014,12,92,...,0.0,111,UNE MAISON,646996.6678122191,6869979.346030179,3231.707317,92004,92004,ASNIÈRES-SUR-SEINE,2014
4,49806524,0b57d2c9a1b36749679e9b97405f9e16,0b57d2c9a1b36749679e9b97405f9e16,1,NaN,NaN,2014-02-12,2014,2,92,...,0.0,121,UN APPARTEMENT,647008.4305120455,6859063.344633167,4333.333333,92040,92040,ISSY-LES-MOULINEAUX,2014


In [3]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [4]:
# ============================================================
# 📊 Widget 1 – Classement des Communes (version corrigée)
# ============================================================

# --- Imports nécessaires ---
import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import display, Markdown

# --- Configuration du renderer Plotly ---
# 💡 Pour Jupyter Notebook ou JupyterLab :
pio.renderers.default = "notebook_connected"
# 👉 Si tu es dans VS Code ou Spyder, remplace par :
# pio.renderers.default = "browser"

# ============================================================
# 📊 Widget 1 – Classement des Communes
# ============================================================

display(Markdown("## 📊 Widget 1 – Classement des Communes"))

# --- Sélection d'une période ---
if df["annee"].notna().sum() > 0:
    annees = sorted(df["annee"].dropna().unique().astype(int))
    print(f"Années disponibles : {annees}")
    annee_debut = int(input(f"👉 Entrez l’année de départ (ex: {annees[0]}): ") or annees[0])
    annee_fin = int(input(f"👉 Entrez l’année de fin (ex: {annees[-1]}): ") or annees[-1])
    df_filtre = df[df["annee"].between(annee_debut, annee_fin)]
else:
    print("⚠️ Aucune donnée temporelle disponible.")
    df_filtre = df

# --- Agrégations par commune ---
agg = (
    df_filtre.groupby("nom_commune")
    .agg(
        prix_median_m2=("prix_m2", "median"),
        volume=("prix_m2", "size"),
        surface_moy=("sbati", "mean")
    )
    .reset_index()
)

# --- Top 20 des communes les plus chères ---
agg = agg.sort_values("prix_median_m2", ascending=False).head(20)

# --- Graphique Plotly ---
fig1 = px.bar(
    agg,
    x="nom_commune",
    y="prix_median_m2",
    color="prix_median_m2",
    color_continuous_scale="Viridis",
    text_auto=".0f",
    title=f"🏆 Top 20 des communes les plus chères ({annee_debut}-{annee_fin})"
)
fig1.update_layout(
    xaxis_title=None,
    yaxis_title="Prix médian au m² (€)",
    title_x=0.5,
    template="plotly_white"
)

# --- Affichage du graphique ---
fig1.show()

# --- Statistiques globales ---
prix_median_global = df_filtre["prix_m2"].median()
nb_ventes = len(df_filtre)
nb_communes = df_filtre["nom_commune"].nunique()

display(Markdown(f"""
**💶 Prix médian au m² (IDF)** : {prix_median_global:,.0f} €  
**🏘️ Nombre de ventes analysées** : {nb_ventes:,}  
**🏙️ Communes couvertes** : {nb_communes:,}  
"""))

# --- Tableau de classement ---
display(Markdown("### 📋 Détails du classement"))
display(
    agg.style.format({
        "prix_median_m2": "{:,.0f} €",
        "surface_moy": "{:,.0f} m²",
        "volume": "{:,.0f}"
    })
)

# --- Interprétation ---
display(Markdown("""
### 🧠 Interprétation rapide
- Les communes en haut du classement sont les plus valorisées en €/m².  
- Une hausse du prix médian sur plusieurs années indique une **gentrification**.  
- Le volume de ventes reflète la **liquidité du marché** : un volume élevé = revente facile.  
"""))


## 📊 Widget 1 – Classement des Communes

Années disponibles : [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]
👉 Entrez l’année de départ (ex: 2014): 2020
👉 Entrez l’année de fin (ex: 2025): 202



**💶 Prix médian au m² (IDF)** : nan €  
**🏘️ Nombre de ventes analysées** : 0  
**🏙️ Communes couvertes** : 0  


### 📋 Détails du classement

,nom_commune,prix_median_m2,volume,surface_moy



### 🧠 Interprétation rapide
- Les communes en haut du classement sont les plus valorisées en €/m².  
- Une hausse du prix médian sur plusieurs années indique une **gentrification**.  
- Le volume de ventes reflète la **liquidité du marché** : un volume élevé = revente facile.  


In [7]:
# ============================================================
# 🏙️ Widget 2 – Comparateur de Communes (version à cases à cocher)
# ============================================================

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import plotly.io as pio

# --- Configuration Plotly ---
pio.renderers.default = "notebook_connected"

display(Markdown("## 🏙️ Widget 2 – Comparateur de Communes"))

# --- Liste des communes disponibles ---
communes_dispo = sorted(df["nom_commune"].dropna().unique())
print(f"{len(communes_dispo)} communes disponibles.")

# --- Création d'une grille de cases à cocher ---
checkboxes = [widgets.Checkbox(value=False, description=commune) for commune in communes_dispo]

# --- Conteneur scrollable ---
checkbox_container = widgets.VBox(checkboxes, layout=widgets.Layout(height="300px", overflow="auto"))

# --- Choix du type de graphique ---
chart_type_widget = widgets.RadioButtons(
    options=["Barres", "Radar"],
    value="Barres",
    description="📊 Type :"
)

# --- Bouton de validation ---
compare_button = widgets.Button(description="Comparer ✅", button_style="success")

# --- Zone d'affichage des résultats ---
output = widgets.Output()

# --- Fonction de comparaison ---
def on_compare_clicked(b):
    with output:
        clear_output()
        communes_sel = [cb.description for cb in checkboxes if cb.value]

        if len(communes_sel) < 2:
            display(Markdown("⚠️ **Veuillez sélectionner au moins deux communes.**"))
            return

        # Agrégations
        agg2 = (
            df[df["nom_commune"].isin(communes_sel)]
            .groupby("nom_commune")
            .agg(
                prix_median_m2=("prix_m2", "median"),
                volume=("prix_m2", "size"),
                surface_moy=("sbati", "mean")
            )
            .reset_index()
        )

        display(Markdown("### 📋 Statistiques comparatives"))
        display(agg2.style.format({
            "prix_median_m2": "{:,.0f} €",
            "surface_moy": "{:,.0f} m²",
            "volume": "{:,.0f}"
        }))

        # --- Graphique ---
        chart_type = chart_type_widget.value
        if chart_type == "Barres":
            fig2 = px.bar(
                agg2.melt(id_vars="nom_commune", var_name="Critère", value_name="Valeur"),
                x="nom_commune", y="Valeur", color="Critère",
                barmode="group", text_auto=True,
                color_discrete_sequence=px.colors.qualitative.Set2,
                title="Comparaison entre les communes"
            )
        else:
            categories = ["prix_median_m2", "volume", "surface_moy"]
            fig2 = go.Figure()
            for _, row in agg2.iterrows():
                fig2.add_trace(go.Scatterpolar(
                    r=[row[c] for c in categories],
                    theta=categories,
                    fill='toself',
                    name=row["nom_commune"]
                ))
            fig2.update_layout(
                polar=dict(radialaxis=dict(visible=True, showticklabels=True)),
                title="Comparaison entre les communes (Radar chart)",
                showlegend=True
            )

        fig2.show()

        # Interprétation
        display(Markdown("""
        ### 🧠 Interprétation rapide
        - **Prix médian au m²** → reflète la tension immobilière (valeurs plus hautes = zones plus chères).  
        - **Volume de ventes** → indique la liquidité du marché (plus de ventes = marché plus actif).  
        - **Surface moyenne** → donne une idée du type de biens dominants (grandes surfaces = zones pavillonnaires).  
        """))

# --- Lier le bouton à la fonction ---
compare_button.on_click(on_compare_clicked)

# --- Affichage du widget ---
display(Markdown("### 🏡 Sélectionnez les communes à comparer"))
display(checkbox_container)
display(chart_type_widget)
display(compare_button)
display(output)


## 🏙️ Widget 2 – Comparateur de Communes

36 communes disponibles.


### 🏡 Sélectionnez les communes à comparer

RadioButtons(description='📊 Type :', options=('Barres', 'Radar'), value='Barres')

Button(button_style='success', description='Comparer ✅', style=ButtonStyle())

Output()